In [6]:
import yaml
import os
import pathlib
import time
from Agent.xwAPI import xwBackend
from utils.construct_prompt import get_api_doc
from utils.ChatGPT import ChatGPT

In [7]:
config_file = "./config/config.yaml"
with open(config_file, mode='r') as file:
    config = yaml.load(file, Loader=yaml.Loader)


interaction_mode = config.get('interaction_mode', False)
agent_config = config["Agent"]
agent_config['ChatGPT_1']['api_keys'] = [os.environ['OPENAI_API_KEY']]
prompt_format = agent_config['ChatGPT_1']['prompt_format']
with open(agent_config['api_doc_path']) as f:
    api_doc = yaml.load(f, Loader=yaml.FullLoader)

In [8]:
if agent_config['API_backend'] == 'xw':
    xw_backend = xwBackend(agent_config['APP_backend'], api_doc)

In [ ]:
path_generator = pathlib.Path(f"../output_dir/refined_responses/").glob("**/*")
response_log_paths = [str(path).split("\\")[-1] for path in path_generator if path.is_file()]
excel_file_names = [f"{log_file[2:-5]}.xlsx" for log_file in response_log_paths]
excel_file_names

In [ ]:
response_log_paths

In [ ]:
index = 1
file = f"../dataset/task_sheets/{excel_file_names[index].replace('yaml', 'xlsx')}"
file

In [6]:
if file is not None:
    time.sleep(0.5)
    xw_backend.OpenWorkbook(file)

In [ ]:
sheet_state = xw_backend.GetSheetsState()
sheet_state

In [8]:
api_list, api_usage, api_detail_doc = get_api_doc(prompt_format, api_doc)

In [9]:
response_log_path = f"../output_dir/refined_responses/{response_log_paths[index]}"
with open(response_log_path) as file:
    response_log = yaml.load(file, Loader=yaml.Loader)

In [ ]:
prompt = """Summarize the each sub-step of refined responses into explanations in natural language:
{refined_response}
Here is the supplementary doc you can reference:
{documentation}
Here is the corresponding sheet state:
{sheet_state}
"""
prompt

In [ ]:
unique_function_names = set([sub_instruction[:sub_instruction.find("(")] for sub_instruction in response_log["refined_response"]])
unique_function_names

In [12]:
documentation_for_functions = {"documentation": [api_detail_doc[name] for name in unique_function_names]}

In [ ]:
prompt = prompt.format(refined_response=yaml.dump(response_log),documentation=yaml.dump(documentation_for_functions), sheet_state=sheet_state)
print(prompt)

In [14]:
async def call_chat_gpt():
    try:
        chatbot = ChatGPT(agent_config['ChatGPT_1'], context=[], interaction_mode=True)
        response = await chatbot(prompt)
    except Exception as e:
        print(f"error occurs when parsing response: {e}")
    else:
        return response


In [ ]:
response = await call_chat_gpt()

In [ ]:
print(response)